In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sellers=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_sellers_dataset.csv")
customers=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_customers_dataset.csv")
geolocation=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_geolocation_dataset.csv")
payments=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_order_payments_dataset.csv")
reviews=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_order_reviews_dataset.csv")
orders=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_orders_dataset.csv")
items=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_order_items_dataset.csv")
products=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/olist_products_dataset.csv")
translation=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/olist/product_category_name_translation.csv")

In [12]:
whole=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/최근 꺼/전처리1차_전체.csv")
state_sp=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/최근 꺼/전처리1차_SP.csv")
state_ba=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/최근 꺼/전처리1차_BA.csv")

# DB SCAN 

## SP

In [ ]:
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# 필요한 컬럼으로 구성
coordinates = state_sp[['geolocation_lat', 'geolocation_lng']]

# DBSCAN 적용
dbscan = DBSCAN(eps=0.01, min_samples=5)  # 파라미터 조정 가능
state_sp['cluster'] = dbscan.fit_predict(coordinates)  # 클러스터 결과 추가

# 클러스터링 결과 확인
print(state_sp['cluster'].value_counts())  # 클러스터 수 확인

# 지도 시각화
# 중심 좌표 설정
map_center = [state_sp['geolocation_lat'].mean(), state_sp['geolocation_lng'].mean()]
map_sp = folium.Map(location=map_center, zoom_start=10)

# 클러스터 시각화를 위한 MarkerCluster 사용
marker_cluster = MarkerCluster().add_to(map_sp)

# 데이터 포인트 추가
for i, row in state_sp.iterrows():
    color = 'blue' if row['cluster'] == -1 else 'green'  # 이상치는 파란색, 정상 클러스터는 초록색
    popup_info = (
        f"Order ID: {row['order_id']}<br>"
        f"Prefix: {row['customer_zip_code_prefix']}"    # 추가 정보
    )

    folium.CircleMarker(
        location=(row['geolocation_lat'], row['geolocation_lng']),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=popup_info  # 팝업 정보 추가
    ).add_to(marker_cluster)

# 지도 출력
map_sp


이상치 제거

In [39]:
state_sp[state_sp['customer_zip_code_prefix'].isin([13375,14760,15367])]

,order_id,order_status,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,time_difference_0,time_difference_1,time_difference_2,time_difference_total,cluster
34502,d978adcd421de6a71e9974afe4414e4f,delivered,5b11c53161333319dc2064ab1049730e,4f2529e3e1328a81898f11bb9d4e281d,13375,mombuca,SP,-20.569366,-46.023119,2018-04-18 16:43:47,2018-04-18 17:14:49,2018-04-19 22:58:38,2018-04-24 21:12:44,2018-05-11,0 days 00:31:02,1 days 05:43:49,4 days 22:14:06,6 days 04:28:57,-1
34503,db2e05029f564b8953ddf3766e3b99f5,delivered,e392266c72da3c0c12da2d2bf0bdc386,9ad7efc09eb9e6f0a7ba95190500dbf6,13375,mombuca,SP,-20.569366,-46.023119,2018-02-15 07:45:24,2018-02-16 07:50:44,2018-02-16 19:32:34,2018-03-23 21:19:00,2018-03-08,1 days 00:05:20,0 days 11:41:50,35 days 01:46:26,36 days 13:33:36,-1
34504,8678b97b317b045765dcbdf85bcb74fb,delivered,068826320c90fa65a4db813cb2e1878f,c6d758568baffc308da80fe73a2a4030,13375,mombuca,SP,-20.569366,-46.023119,2017-11-29 14:21:01,2017-11-29 14:34:31,2017-12-01 18:38:36,2017-12-08 12:02:59,2017-12-19,0 days 00:13:30,2 days 04:04:05,6 days 17:24:23,8 days 21:41:58,-1
40252,4c612311c4c424c8ccf682de9b443659,delivered,78337577a7b3160393cf5f5e69f13d52,2dd1d99c3ff427b74f938c2daf370622,14760,ibitiuva,SP,-19.886871,-47.016357,2018-08-09 06:35:43,2018-08-10 03:24:35,2018-08-10 15:10:00,2018-08-16 00:28:30,2018-08-27,0 days 20:48:52,0 days 11:45:25,5 days 09:18:30,6 days 17:52:47,-1
41085,dc1b1cb0a0bef9a1b53b35be77247cb3,delivered,ef7c85f2ad75eed987fd08a601715fd9,a898766c42344d7b1b9875bf6440c2ff,15367,bandeirantes doeste,SP,-23.106241,-50.377957,2018-02-21 11:45:27,2018-02-23 02:31:11,2018-02-23 22:16:35,2018-03-06 22:57:22,2018-03-19,1 days 14:45:44,0 days 19:45:24,11 days 00:40:47,13 days 11:11:55,-1


In [40]:
# 드랍할 order_id 목록
order_ids_to_drop = ['d978adcd421de6a71e9974afe4414e4f', 'db2e05029f564b8953ddf3766e3b99f5','8678b97b317b045765dcbdf85bcb74fb','4c612311c4c424c8ccf682de9b443659','dc1b1cb0a0bef9a1b53b35be77247cb3']

# 드랍할 행을 제외한 새로운 DataFrame 생성
state_sp_cleaned = state_sp[~state_sp['order_id'].isin(order_ids_to_drop)]


바뀐지 확인

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# 2. 필요한 컬럼으로 구성
coordinates = state_sp2_cleaned[['geolocation_lat', 'geolocation_lng']]

# 3. DBSCAN 적용
dbscan = DBSCAN(eps=0.01, min_samples=5)  # 파라미터 조정 가능
state_sp2_cleaned['cluster'] = dbscan.fit_predict(coordinates)  # 클러스터 결과 추가

# 4. 지도 시각화
map_center = [state_sp2_cleaned['geolocation_lat'].mean(), state_sp2_cleaned['geolocation_lng'].mean()]
map_sp = folium.Map(location=map_center, zoom_start=10)

# 클러스터 시각화를 위한 MarkerCluster 사용
marker_cluster = MarkerCluster().add_to(map_sp)

# 데이터 포인트 추가
for i, row in state_sp2_cleaned.iterrows():
    color = 'blue' if row['cluster'] == -1 else 'green'  # 이상치는 파란색, 정상 클러스터는 초록색
    popup_info = (
        f"Order ID: {row['order_id']}"  # 추가 정보
    )

    folium.CircleMarker(
        location=(row['geolocation_lat'], row['geolocation_lng']),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=popup_info  # 팝업 정보 추가
    ).add_to(marker_cluster)

# 5. 지도 출력
map_sp


완료

In [41]:
# CSV 파일로 저장
state_sp_cleaned.to_csv('C:/Users/6aimh/OneDrive/바탕 화면/마케팅/최근 꺼/SP최근.csv', index=False) 

In [42]:
state_sp_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41107 entries, 0 to 41111
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       41107 non-null  object 
 1   order_status                   41107 non-null  object 
 2   customer_id                    41107 non-null  object 
 3   customer_unique_id             41107 non-null  object 
 4   customer_zip_code_prefix       41107 non-null  int64  
 5   customer_city                  41107 non-null  object 
 6   customer_state                 41107 non-null  object 
 7   geolocation_lat                41107 non-null  float64
 8   geolocation_lng                41107 non-null  float64
 9   order_purchase_timestamp       41107 non-null  object 
 10  order_approved_at              41101 non-null  object 
 11  order_delivered_carrier_date   40808 non-null  object 
 12  order_delivered_customer_date  40475 non-null 

## BA

In [30]:
state_ba=pd.read_csv("C:/Users/6aimh/OneDrive/바탕 화면/마케팅/.venv/입지선정 전처리/전처리1차_입지선정_BA.csv")
state_ba.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng,time_difference_0,time_difference_1,time_difference_2,time_difference_total
0,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.177924,-44.660711,1 days 06:42:50,0 days 11:06:33,12 days 00:56:45,13 days 18:46:08
1,83018ec114eee8641c97e08f7b4e926f,7f8c8b9c2ae27bf3300f670c3d478be8,delivered,2017-10-26 15:54:26,2017-10-26 16:08:14,2017-10-26 21:46:53,2017-11-08 22:22:00,2017-11-23,634f09f6075fe9032e6c19609ffe995a,44024,feira de santana,BA,-12.244274,-38.979613,0 days 00:13:48,0 days 05:38:39,13 days 00:35:07,13 days 06:27:34
2,acce194856392f074dbf9dada14d8d82,7e20bf5ca92da68200643bda76c504c6,delivered,2018-06-04 00:00:13,2018-06-05 00:35:10,2018-06-05 13:24:00,2018-06-16 15:20:55,2018-07-18,576ea0cab426cd8a00fad9a9c90a4494,41213,salvador,BA,-12.939062,-38.438353,1 days 00:34:57,0 days 12:48:50,11 days 01:56:55,12 days 15:20:42
3,d22e9fa5731b9e30e8b27afcdc2f8563,756fb9391752dad934e0fe3733378e57,delivered,2018-08-04 23:25:30,2018-08-04 23:35:13,2018-08-06 15:03:00,2018-08-13 23:34:42,2018-09-13,394b2ce444baae9ae609f5d32000de0f,47850,luis eduardo magalhaes,BA,-12.092513,-45.806927,0 days 00:09:43,1 days 15:27:47,7 days 08:31:42,9 days 00:09:12
4,d17dc4a904426827ca80f2ccb3a6be56,569cf68214806a39acc0f39344aea67f,delivered,2017-05-14 20:28:25,2017-05-14 20:42:45,2017-05-16 08:17:46,2017-05-25 09:14:31,2017-06-12,c2551ea089b7ebbc67a2ea8757152514,44380,cruz das almas,BA,-12.670246,-39.103277,0 days 00:14:20,1 days 11:35:01,9 days 00:56:45,10 days 12:46:06


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# 2. 필요한 컬럼으로 구성
coordinates = state_ba[['geolocation_lat', 'geolocation_lng']]

# 3. DBSCAN 적용
dbscan = DBSCAN(eps=0.01, min_samples=5)  # 파라미터 조정 가능
state_ba['cluster'] = dbscan.fit_predict(coordinates)  # 클러스터 결과 추가

# 4. 지도 시각화
map_center = [state_ba['geolocation_lat'].mean(), state_ba['geolocation_lng'].mean()]
map_ba = folium.Map(location=map_center, zoom_start=10)

# 클러스터 시각화를 위한 MarkerCluster 사용
marker_cluster = MarkerCluster().add_to(map_ba)

# 데이터 포인트 추가
for i, row in state_ba.iterrows():
    color = 'blue' if row['cluster'] == -1 else 'green'  # 이상치는 파란색, 정상 클러스터는 초록색
    popup_info = (
        f"Order ID: {row['order_id']}"  # 추가 정보
    )

    folium.CircleMarker(
        location=(row['geolocation_lat'], row['geolocation_lng']),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=popup_info  # 팝업 정보 추가
    ).add_to(marker_cluster)

# 5. 지도 출력
map_ba


이상치 제거

In [32]:
#  드랍할 order_id 목록
order_ids_to_drop = ['ed7774b614a7ad220649f09dc6a4c043','897ec6416d50126a9061626f0fc2d658','24161cb9857591fc2fe821db0c4dd3b0']

# 드랍할 행을 제외한 새로운 DataFrame 생성
state_ba = state_ba[~state_ba['order_id'].isin(order_ids_to_drop)]

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# 2. 필요한 컬럼으로 구성
coordinates = state_ba[['geolocation_lat', 'geolocation_lng']]

# 3. DBSCAN 적용
dbscan = DBSCAN(eps=0.01, min_samples=5)  # 파라미터 조정 가능
state_ba['cluster'] = dbscan.fit_predict(coordinates)  # 클러스터 결과 추가

# 4. 지도 시각화
map_center = [state_ba['geolocation_lat'].mean(), state_ba['geolocation_lng'].mean()]
map_ba = folium.Map(location=map_center, zoom_start=10)

# 클러스터 시각화를 위한 MarkerCluster 사용
marker_cluster = MarkerCluster().add_to(map_ba)

# 데이터 포인트 추가
for i, row in state_ba.iterrows():
    color = 'blue' if row['cluster'] == -1 else 'green'  # 이상치는 파란색, 정상 클러스터는 초록색
    popup_info = (
        f"Order ID: {row['order_id']}"  # 추가 정보
    )

    folium.CircleMarker(
        location=(row['geolocation_lat'], row['geolocation_lng']),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=popup_info  # 팝업 정보 추가
    ).add_to(marker_cluster)

# 5. 지도 출력
map_ba


In [35]:
# CSV 파일로 저장
state_ba.to_csv('C:/Users/6aimh/OneDrive/바탕 화면/마케팅/최근 꺼/BA최근.csv', index=False) 